In [52]:
import os
import re
cwd = os.getcwd()
if re.search("protein-reconstruction.+", cwd):
    os.chdir("..")
print(os.getcwd())
import torch
from torch_geometric.loader import DataLoader
from preprocessing.dataset import load_dataset
from models.pretraining.graph_infomax import DeepGraphInfomaxV2, RandomPermutationCorruption, MeanPoolReadout
from models.pretraining.encoders import RevSAGEConvEncoder
from tqdm.autonotebook import tqdm
from torchinfo import summary
from typing import final

/home/prot_prj/PycharmProjects/protein-reconstruction


In [53]:
ds_train = load_dataset("data/cleaned/pscdb/train")
ds_val = load_dataset("data/cleaned/pscdb/validation")
dl_train = DataLoader(ds_train, batch_size=1, shuffle=True)
dl_val = DataLoader(ds_val, batch_size=1, shuffle=True)
print(ds_train[0], ds_val[0])

Data(edge_index=[2, 1616], node_id=[858], coords=[858, 3], meiler=[858, 7], name=[1], dist_mat=[1], num_nodes=858, graph_y=5, x=[858, 10], y=5) Data(edge_index=[2, 1721], node_id=[901], coords=[901, 3], meiler=[901, 7], name=[1], dist_mat=[1], num_nodes=901, graph_y=5, x=[901, 10], y=5)


In [54]:
MODEL_PATH_STATE_DICT: final = "data/fitted/pretraining/dgi/dgi_rev_sage_test5/checkpoint.pt"
MODEL_PATH_PARAMS: final = "data/fitted/pretraining/dgi/dgi_rev_sage_test5/constructor_params.pt"


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
state_dict = torch.load(MODEL_PATH_STATE_DICT)
constructor_params = torch.load(MODEL_PATH_PARAMS)
dgi = DeepGraphInfomaxV2.from_constructor_params(constructor_params, encoder_constructor=RevSAGEConvEncoder, readout=MeanPoolReadout(device, sigmoid=False), corruption=RandomPermutationCorruption(device))
print(summary(dgi, device=device, depth=5))

Layer (type:depth-idx)                                       Param #
DeepGraphInfomaxV2                                           2,500
├─RevSAGEConvEncoder: 1-1                                    --
│    └─Linear: 2-1                                           550
│    └─LayerNorm: 2-2                                        100
│    └─ModuleList: 2-3                                       --
│    │    └─GroupAddRev: 3-1                                 --
│    │    │    └─ModuleList: 4-1                             --
│    │    │    │    └─SAGEConvBlock: 5-1                     65
│    │    │    │    └─SAGEConvBlock: 5-2                     65
│    │    │    │    └─SAGEConvBlock: 5-3                     65
│    │    │    │    └─SAGEConvBlock: 5-4                     65
│    │    │    │    └─SAGEConvBlock: 5-5                     65
│    │    │    │    └─SAGEConvBlock: 5-6                     65
│    │    │    │    └─SAGEConvBlock: 5-7                     65
│    │    │    │    └─SAGEConv

In [55]:
x_list = []
y_list = []
dgi.eval()
dgi = dgi.to(device)
with torch.no_grad():
    for el in tqdm(iter(dl_train)):
        el = el.to(device)
        pos_z, neg_z, summary = dgi(el.x, el.edge_index)
        x_list.append(summary)
        y_list.append(el.y)
    print("Done train conversion.")

  0%|          | 0/550 [00:00<?, ?it/s]

Done train.


In [56]:
x_list_val = []
y_list_val = []
with torch.no_grad():
    for el in tqdm(iter(dl_val)):
        el = el.to(device)
        pos_z, neg_z, summary = dgi(el.x, el.edge_index)
        x_list_val.append(summary)
        y_list_val.append(el.y)
    print("Done validation conversion.")
dgi_2 = dgi.to('cpu')  # free GPU memory
del dgi
dgi = dgi_2

  0%|          | 0/119 [00:00<?, ?it/s]

Done validation.


In [66]:
max_len = 0
for i in range(0, len(x_list)):
    if x_list[i].shape[0] > max_len:
        max_len = x_list[i].shape[0]
for i in range(0, len(x_list_val)):
    if x_list_val[i].shape[0] > max_len:
        max_len = x_list_val[i].shape[0]

# Mean embeddings if required
for i in tqdm(range(0, len(x_list))):
    if len(x_list[i].shape) > 1:
        x_list[i] = torch.mean(x_list[i], dim=0, keepdim=False)
    #x_list[i] = torch.nn.functional.normalize(x_list[i], p=2.0, dim=0, eps=1e-12)

for i in tqdm(range(0, len(x_list_val))):
    if len(x_list_val[i].shape) > 1:
        x_list_val[i] = torch.mean(x_list_val[i], dim=0, keepdim=False)
    #x_list[i] = torch.nn.functional.normalize(x_list[i], p=2.0, dim=0, eps=1e-12)

print(x_list[0].shape)
print(x_list_val[0].shape)
x_train = torch.stack(x_list)
y_train = torch.tensor(y_list)
x_val = torch.stack(x_list_val)
y_val = torch.tensor(y_list_val)
print(x_train.shape)
print(y_train.shape)
print("------------------------------------")
print(x_val.shape)
print(y_val.shape)

  0%|          | 0/550 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

torch.Size([50])
torch.Size([50])
torch.Size([550, 50])
torch.Size([550])
------------------------------------
torch.Size([119, 50])
torch.Size([119])


In [73]:

scores = dgi.test(train_z=x_train, train_y=y_train, test_z=x_val, test_y=y_val, solver='newton-cg', max_iter=1000)
print(f"Newton-Cg Score: {scores}")
scores = dgi.test(train_z=x_train, train_y=y_train, test_z=x_val, test_y=y_val, solver='liblinear', max_iter=1000)
print(f"liblinear Score: {scores}")
#scores = dgi.test(train_z=x_train, train_y=y_train, test_z=x_val, test_y=y_val, solver='newton-cholesky')
#print(f"newton-cholesky Score: {scores}")
scores = dgi.test(train_z=x_train, train_y=y_train, test_z=x_val, test_y=y_val, solver='sag', max_iter=1000)
print(f"sag Score: {scores}")
scores = dgi.test(train_z=x_train, train_y=y_train, test_z=x_val, test_y=y_val, solver='saga', max_iter=1000)
print(f"saga Score: {scores}")
scores = dgi.test(train_z=x_train, train_y=y_train, test_z=x_val, test_y=y_val, solver='lbfgs', max_iter=1000)
print(f"lbfgs Score: {scores}")

Newton-Cg Score: 0.3865546218487395
liblinear Score: 0.3865546218487395
sag Score: 0.3865546218487395
saga Score: 0.3865546218487395
lbfgs Score: 0.3865546218487395


/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


In [131]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(50, 50), verbose=True, warm_start=False, early_stopping=True, learning_rate='adaptive', max_iter=300, validation_fraction=0.02, n_iter_no_change=170, solver='adam',
                    tol=1e-30)
mlp.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
print(mlp.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy()))

Iteration 1, loss = 2.05836798
Validation score: 0.000000
Iteration 2, loss = 1.89421475
Validation score: 0.000000
Iteration 3, loss = 1.78238684
Validation score: 0.000000
Iteration 4, loss = 1.71031610
Validation score: 0.090909
Iteration 5, loss = 1.67092058
Validation score: 0.090909
Iteration 6, loss = 1.64363641
Validation score: 0.090909
Iteration 7, loss = 1.62920071
Validation score: 0.090909
Iteration 8, loss = 1.61702020
Validation score: 0.181818
Iteration 9, loss = 1.60619582
Validation score: 0.181818
Iteration 10, loss = 1.59821643
Validation score: 0.181818
Iteration 11, loss = 1.59155058
Validation score: 0.181818
Iteration 12, loss = 1.58463251
Validation score: 0.181818
Iteration 13, loss = 1.57816571
Validation score: 0.181818
Iteration 14, loss = 1.57303924
Validation score: 0.181818
Iteration 15, loss = 1.56853618
Validation score: 0.090909
Iteration 16, loss = 1.56479977
Validation score: 0.090909
Iteration 17, loss = 1.56087352
Validation score: 0.090909
Iterat

In [211]:
from sklearn.svm import SVC

kernels = ["rbf", "linear", "sigmoid", "poly"]
decision_shapes = ["ovr", "ovo"]
penalties = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 1.5]
degrees = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 25, 30, 40, 50]
max_acc: float = 0.0
best_params = {"kernel": None, "C": None, "decision_shape": None, "degree": None}


for kernel in tqdm(kernels):
    for decision_shape in decision_shapes:
        for c in penalties:
            if kernel == "poly":
                for degree in penalties:
                    svm = SVC(kernel=kernel, decision_function_shape=decision_shape, C=c, degree=degree, random_state=47)
                    svm.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
                    acc = svm.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy())

                    if acc > max_acc:
                        best_params["kernel"] = kernel
                        best_params["C"] = c
                        best_params["decision_shape"] = decision_shape
                        best_params["degree"] = degree
            else:
                svm = SVC(kernel=kernel, decision_function_shape=decision_shape, C=c, random_state=47)
                svm.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
                acc = svm.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy())

                if acc > max_acc:
                    max_acc = acc
                    best_params["kernel"] = kernel
                    best_params["C"] = c
                    best_params["decision_shape"] = decision_shape
                    best_params["degree"] = None

print(f"SVM: best_params: {best_params}, accuracy: {max_acc}")

  0%|          | 0/4 [00:00<?, ?it/s]

SVM: best_params: {'kernel': 'rbf', 'C': 0.0001, 'decision_shape': 'ovr', 'degree': None}, accuracy: 0.4369747899159664


In [175]:
from sklearn.neighbors import KNeighborsClassifier

max_acc_k = 2
max_acc: float = 0.0
max_neighbors = 150
for k in tqdm(range(2, max_neighbors)):
    knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    knn.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
    acc = knn.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy())

    if acc > max_acc:
        max_acc = acc
        max_acc_k = k
    print(f"k: {k}, score: {acc}")

print(f"Best k: {max_acc_k}, best score: {max_acc}")

  0%|          | 0/148 [00:00<?, ?it/s]

k: 2, score: 0.14285714285714285
k: 3, score: 0.19327731092436976
k: 4, score: 0.20168067226890757
k: 5, score: 0.19327731092436976
k: 6, score: 0.21008403361344538
k: 7, score: 0.23529411764705882
k: 8, score: 0.2773109243697479
k: 9, score: 0.2773109243697479
k: 10, score: 0.31932773109243695
k: 11, score: 0.2773109243697479
k: 12, score: 0.31932773109243695
k: 13, score: 0.31932773109243695
k: 14, score: 0.3277310924369748
k: 15, score: 0.33613445378151263
k: 16, score: 0.3445378151260504
k: 17, score: 0.35294117647058826
k: 18, score: 0.3445378151260504
k: 19, score: 0.35294117647058826
k: 20, score: 0.35294117647058826
k: 21, score: 0.3949579831932773
k: 22, score: 0.3697478991596639
k: 23, score: 0.3697478991596639
k: 24, score: 0.3697478991596639
k: 25, score: 0.37815126050420167
k: 26, score: 0.36134453781512604
k: 27, score: 0.36134453781512604
k: 28, score: 0.37815126050420167
k: 29, score: 0.35294117647058826
k: 30, score: 0.36134453781512604
k: 31, score: 0.3697478991596639

In [180]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel, PairwiseKernel


n_restarts = [0, 5, 10, 15, 20, 25]
kernels = [RBF(), WhiteKernel(), ConstantKernel(), PairwiseKernel()]
warm_starts = [True, False]
max_acc = 0.0
best_params = {"n_restarts": None, "warm_start": None, "kernel": None}

for kernel in kernels:
    for n in n_restarts:
        for ws in warm_starts:
            gpc = GaussianProcessClassifier(kernel=kernel, n_restarts_optimizer=n, max_iter_predict=100, warm_start=ws, random_state=47, n_jobs=-1)
            gpc.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
            acc = gpc.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy())
            if acc > max_acc:
                max_acc = acc
                best_params["n_restarts"] = n
                best_params["warm_start"] = ws
                best_params["kernel"] = kernel
            print(f"Accuracy: {acc}, warm_starts: {ws}, n_restarts: {n}, kernel: {kernel}")
print(f"Best params: {best_params}, best accuracy: {max_acc}")

/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/a

Accuracy: 0.4369747899159664, warm_starts: True, n_restarts: 0, kernel: RBF(length_scale=1)


/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/a

Accuracy: 0.4369747899159664, warm_starts: False, n_restarts: 0, kernel: RBF(length_scale=1)


/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/a

...

Accuracy: 0.3865546218487395, warm_starts: True, n_restarts: 15, kernel: PairwiseKernel(gamma=1.0, metric=linear)
Accuracy: 0.3865546218487395, warm_starts: False, n_restarts: 15, kernel: PairwiseKernel(gamma=1.0, metric=linear)


/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter gamma is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter gamma is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter gamma is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Accuracy: 0.3865546218487395, warm_starts: True, n_restarts: 20, kernel: PairwiseKernel(gamma=1.0, metric=linear)
Accuracy: 0.3865546218487395, warm_starts: False, n_restarts: 20, kernel: PairwiseKernel(gamma=1.0, metric=linear)


/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter gamma is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter gamma is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter gamma is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/prot_prj/anaconda3/envs/prot_pr

Accuracy: 0.3865546218487395, warm_starts: True, n_restarts: 25, kernel: PairwiseKernel(gamma=1.0, metric=linear)
Accuracy: 0.3865546218487395, warm_starts: False, n_restarts: 25, kernel: PairwiseKernel(gamma=1.0, metric=linear)
Best params: {'n_restarts': 0, 'warm_start': True, 'kernel': RBF(length_scale=1)}, best accuracy: 0.4369747899159664


In [183]:
from sklearn.naive_bayes import GaussianNB

nbc = GaussianNB()
nbc.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
acc = nbc.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy())
print(acc)

0.14285714285714285


In [207]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

solvers = ["svd", "lsqr", "eigen"]
reg_params = [0.0, 0.01, 0.05, 0.1, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1.0, 1.1]
max_acc = 0.0
best_params = {"solver": None}

for solver in solvers:
    lda = LinearDiscriminantAnalysis(solver=solver)
    lda.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
    acc = lda.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy())

    if acc > max_acc:
        best_params["solver"] = solver
        max_acc = acc
print(f"Linear discriminant analysis: best params: {best_params}, best accuracy: {max_acc}")

best_params = {"reg_param": None}
max_acc = 0.0
for reg in reg_params:
    qda = QuadraticDiscriminantAnalysis(reg_param=reg)
    qda.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
    acc = qda.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy())
    if acc > max_acc:
        best_params["reg_param"] = reg
        max_acc = acc
print(f"Quadratic discriminant analysis: best params: {best_params}, best accuracy: {max_acc}")

Linear discriminant analysis: best params: {'solver': 'lsqr'}, best accuracy: 0.33613445378151263
Quadratic discriminant analysis: best params: {'reg_param': 0.5}, best accuracy: 0.3445378151260504


/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/prot_prj/anaconda3/envs/prot_proj/lib/python3.9/site-packages/sk

In [194]:
from sklearn.tree import DecisionTreeClassifier


criterions = ["gini", "entropy"]
max_depths = [None, 5, 6, 7, 8, 9, 10, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100, 150]
best_params = {"criterion": None, "max_depth": None}
max_acc = 0.0

for max_depth in tqdm(max_depths):
    for criterion in criterions:
        dtc = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)
        dtc.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
        acc = dtc.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy())
        if acc > max_acc:
            best_params["criterion"] = criterion
            best_params["max_depth"] = max_depth
            max_acc = acc
print(f"Decision tree: best params: {best_params}, best accuracy: {max_acc}")

  0%|          | 0/18 [00:00<?, ?it/s]

Decision tree: best params: {'criterion': 'entropy', 'max_depth': 5}, best accuracy: 0.37815126050420167


In [208]:
from sklearn.ensemble import RandomForestClassifier


estimators = [2, 3, 4, 5, 10, 20, 30, 40, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
criterions = ["gini", "entropy"]
max_depths = [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 25, 30, 50]
best_params = {"n_estimators": None, "criterion": None, "max_depth": None}
max_acc = 0.0

for n in tqdm(estimators):
    for max_depth in max_depths:
        for criterion in criterions:
            rfc = RandomForestClassifier(n_estimators=n, criterion=criterion, max_depth=max_depth, random_state=47)
            rfc.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
            acc = rfc.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy())
            if acc > max_acc:
                best_params["criterion"] = criterion
                best_params["max_depth"] = max_depth
                best_params["n_estimators"] = n
                max_acc = acc
print(f"Random Forest: best params: {best_params}, best accuracy: {max_acc}")

  0%|          | 0/18 [00:00<?, ?it/s]

Random Forest: best params: {'n_estimators': 4, 'criterion': 'gini', 'max_depth': 5}, best accuracy: 0.453781512605042


In [202]:
from sklearn.ensemble import AdaBoostClassifier


estimators = [2, 3, 4, 5, 10, 20, 30, 40, 50, 100, 150, 200, 250]
criterions = ["gini", "entropy"]
max_depths = [None, 1, 2, 3, 5, 6, 7, 8, 9, 10]
best_params = {"n_estimators": None, "criterion": None, "max_depth": None}
max_acc = 0.0

for n in tqdm(estimators):
    for max_depth in max_depths:
        for criterion in criterions:
            dtc = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)
            adaboost = AdaBoostClassifier(base_estimator=dtc, n_estimators=n, random_state=47)
            adaboost.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
            acc = adaboost.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy())
            if acc > max_acc:
                best_params["criterion"] = criterion
                best_params["max_depth"] = max_depth
                best_params["n_estimators"] = n
                max_acc = acc
print(f"AdaBoost: best params: {best_params}, best accuracy: {max_acc}")

  0%|          | 0/13 [00:00<?, ?it/s]

AdaBoost: best params: {'n_estimators': 3, 'criterion': 'gini', 'max_depth': 1}, best accuracy: 0.44537815126050423


In [205]:
from sklearn.ensemble import GradientBoostingClassifier


estimators = [2, 3, 4, 5, 10, 20, 30, 40, 50, 100, 150, 200, 250]
criterions = ["friedman_mse", "squared_error"]
losses = ["deviance"]
max_depths = [None, 1, 2, 3, 5, 6, 7, 8, 9, 10]
best_params = {"n_estimators": None, "criterion": None, "max_depth": None}
max_acc = 0.0

for n in tqdm(estimators):
    for max_depth in max_depths:
        for criterion in criterions:
            adaboost = GradientBoostingClassifier(n_estimators=n, criterion=criterion, max_depth=max_depth, random_state=47)
            adaboost.fit(x_train.detach().cpu().numpy(), y_train.detach().cpu().numpy())
            acc = adaboost.score(x_val.detach().cpu().numpy(), y_val.detach().cpu().numpy())
            if acc > max_acc:
                best_params["criterion"] = criterion
                best_params["max_depth"] = max_depth
                best_params["n_estimators"] = n
                max_acc = acc
print(f"GradientBoost: best params: {best_params}, best accuracy: {max_acc}")

  0%|          | 0/13 [00:00<?, ?it/s]

GradientBoost: best params: {'n_estimators': 2, 'criterion': 'friedman_mse', 'max_depth': 1, 'loss': None}, best accuracy: 0.4369747899159664
